# Moving Average Filter C++
<p align="left">
<img src="./img/businesscard.jpg" width="500px" alt="Business Card" align="left" >
</p>
<br>

[www.rptecnologias.com](http://www.rptecnologias.com)
<br>
guitars.ai@rptecnologias.com


In [1]:
// Includes
#include <vector>
#include <random>
#include <iostream>
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include "xplot/xtooltip.hpp"
#include "xplot/xtoolbar.hpp"
#include "xwidgets/xbox.hpp"
#include "xwidgets/xbutton.hpp"
#include "xwidgets/xtogglebutton.hpp"
#include "./Cpp/convolution.hpp"
#pragma cling load("./Cpp/libconvolution.so")

using namespace std;

In [2]:
// Define Moving Average Filter Calculator
std::vector<double> MovingAverage(int taps)
{
    std::vector<double> h;
    for (int i=0;i< taps;i++)
    {
        h.push_back(1.0/taps);
    }
    return h;
}

In [3]:
std::vector<double> h_moving_average = MovingAverage(17);
h_moving_average

{ 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235, 0.0588235 }

In [4]:
// Signals
double x[100]={};
double x_noise[100]={};
random_device rd; 
default_random_engine e(rd()); 
uniform_real_distribution<> d(-1, 1); 
for (int i=0;i<100;i++)
{
    x[i]=i;
    x_noise[i]= i + 2.0*d(e);
}

std::vector<double>x_vec(x, x + sizeof x / sizeof x[0]);
std::vector<double>xnoise_vec(x_noise, x_noise + sizeof x_noise / sizeof x_noise[0]);

In [5]:
// Convolution x_noise * h_moving_average
vector<double> filtered_signal = Convolution1D_Vec(xnoise_vec,h_moving_average, false);
vector<double> filtered_signal_nodelay = Convolution1D_Vec(xnoise_vec,h_moving_average, true);

In [6]:
// Plot Signal

// Tooltips
xpl::tooltip def_tt;
def_tt.fields = std::vector<xtl::xoptional<std::string>>{"x", "y"};
def_tt.formats = std::vector<xtl::xoptional<std::string>>{".2f", ".2f"};

xpl::linear_scale sx, sy;

xpl::lines line_ramp(sx, sy);
line_ramp.x = x_vec;
line_ramp.y = x_vec;
line_ramp.display_legend=true;
line_ramp.labels= std::vector<std::string>{"Ramp"};

xpl::lines line_filtered(sx, sy);
line_filtered.x = x_vec;
line_filtered.y = filtered_signal;
line_filtered.display_legend=true;
line_filtered.labels= std::vector<std::string>{"Filtered Signal"};
line_filtered.colors=std::vector<std::string>{"red"};

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .label("y")
    .orientation("vertical")
    .side("left")
    .finalize();
auto scatter1 = xpl::scatter_generator(sx, sy)
   .x(x_vec)
   .y(xnoise_vec)
    .default_size(8)
    .colors(std::vector<xtl::xoptional<std::string>>{"green"})
    .labels(std::vector<std::string>{"Ramp+Noise"})
    .display_legend(true)
    .tooltip(def_tt)
   .finalize();

auto fig1 = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.025)
    .title("Moving Average Filter")
    .legend_location("bottom_right")
    .finalize();

fig1.add_mark(scatter1);
fig1.add_axis(ax_x);
fig1.add_axis(ax_y);
fig1.add_mark(line_ramp);
fig1.add_mark(line_filtered);
fig1.animation_duration = 1000;
xpl::toolbar toolbar(fig1);
xw::vbox b;
bool delay_correct = false;

In [7]:
void button_click()
{
    delay_correct ? line_filtered.y=filtered_signal: line_filtered.y=filtered_signal_nodelay;
    delay_correct = !delay_correct;
}

// Button
xw::button bt;
bt.on_click(button_click);
bt.description = "Delay";


b.add(fig1);
b.add(toolbar);
b.add(bt);
b


A Jupyter widget